In [1]:
import os
import random
import numpy as np
import torch
import torchvision
from torchsummary import summary

from fpga_nn_backend.datasets import *
from fpga_nn_backend.training import *
from fpga_nn_backend.evaluation import *
from fpga_nn_backend.models.relu_toy_models import *
from fpga_nn_backend.quantization import *
from fpga_nn_backend.fpga_simple.emulation import *
from fpga_nn_backend.fpga_simple.conversion import *
from fpga_nn_backend.utils import *

In [2]:
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
# Detect if we have a GPU available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")

PyTorch Version: 1.10.0
Torchvision Version: 0.11.1


In [3]:
data_dir = get_rel_pkg_path("dataset/")
weights_dir = get_rel_pkg_path("weights/")
session_dir = get_rel_pkg_path("sessions/")
models_dir = get_rel_pkg_path("models/")

In [4]:
dataset_type = ImageDatasetType.MNIST

In [5]:
orig_datasets = get_img_dataset(data_dir, dataset_type)

In [6]:
datasets = apply_img_transforms(orig_datasets, dataset_type, flatten=True)

In [7]:
dataloaders = get_dataloaders(datasets, 128, 128, num_workers=0)

In [8]:
input_dim = IMG_DATASET_TO_IMG_SIZE_FLAT[dataset_type]
num_classes = IMG_DATASET_TO_NUM_CLASSES[dataset_type]

model = ReLUToyModel(input_dim, num_classes, layer_dims=[])
model = model.to(device)

In [9]:
model = QuantWrapper(model)

In [10]:
summary(model, (input_dim,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         QuantStub-1                  [-1, 784]               0
            Linear-2                   [-1, 10]           7,840
      ReLUToyModel-3                   [-1, 10]               0
       DeQuantStub-4                   [-1, 10]               0
Total params: 7,840
Trainable params: 7,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------


In [11]:
# model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 11-18-2021 11-04-36 PM\Weights Best.pckl")))

In [12]:
# model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 11-20-2021 06-39-40 PM\Weights Best.pckl")))

In [13]:
model.load_state_dict(torch.load(os.path.join(weights_dir, r"Experiment 12-07-2021 11-47-53 PM\Weights Best.pckl")))

<All keys matched successfully>

In [14]:
criterion = get_loss()
criterion = criterion.to(device)

In [15]:
model.eval()
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
#torch.quantization.fuse_modules(model.model.layers, [['0', '1'], ['2', '3'], ['4', '5'], ['6', '7']], inplace=True)
model = torch.quantization.prepare(model)
stats = get_dataloader_stats(dataloaders['test'], model, criterion, device)
model_int8 = torch.quantization.convert(model)

C:\Users\Shahir\anaconda3\envs\py3-dl\lib\site-packages\torch\ao\quantization\observer.py:172: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:02<00:00, 31.15it/s]
C:\Users\Shahir\anaconda3\envs\py3-dl\lib\site-packages\torch\ao\quantization\observer.py:886: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
C:\Users\Shahir\anaconda3\envs\py3-dl\lib\site-packages\torch\ao\quantiza

In [16]:
print("Accuracy:", stats['acc'])

Accuracy: 0.9034


In [17]:
model_int8

QuantWrapper(
  (model): ReLUToyModel(
    (layers): Sequential(
      (0): QuantizedLinear(in_features=784, out_features=10, scale=0.20200112462043762, zero_point=59, qscheme=torch.per_channel_affine)
    )
  )
  (quant): Quantize(scale=tensor([0.0079]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
)

In [18]:
model_int8.model.layers[0].weight().int_repr()

tensor([[  7, -11, -13,  ...,  -6,   9,   6],
        [ -4,  -8,  -3,  ...,  -6,  -3,  -5],
        [ 14, -14, -14,  ...,   0,  -1,  10],
        ...,
        [ -5, -11,   7,  ...,  -3,   8,  -9],
        [ 16,   7,  12,  ...,  -4,  12, -13],
        [ -8,   2,   3,  ...,   7,  10,   0]], dtype=torch.int8)

In [19]:
stats = get_dataloader_stats(dataloaders['test'], model_int8, criterion, device)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:02<00:00, 34.79it/s]


In [20]:
print("Accuracy:", stats['acc'])

Accuracy: 0.9037


In [21]:
def safe_get_param(param):
    if param is not None:
        return param.int_repr().numpy()
    else:
        return None

In [22]:
converted_nn = ConvertedNN((1, 28, 28))

converted_nn.add_flatten_layer((1, 28, 28), 0, 0)

converted_nn.add_dense_layer((784,), (10,), 0, 0,
    weight=safe_get_param(model_int8.model.layers[0].weight()),
    bias=safe_get_param(model_int8.model.layers[0].bias()))

converted_nn.add_output_layer((10,), 0, 0)

In [23]:
model_int8.model.layers[0]

QuantizedLinear(in_features=784, out_features=10, scale=0.20200112462043762, zero_point=59, qscheme=torch.per_channel_affine)

In [24]:
converted_nn.finalize()

In [25]:
converted_nn.get_layer_info()

{'layers': [{'layer_type': <ConverterLayerType.FLATTEN: 5>,
   'input_shapes': ((1, 28, 28),),
   'output_shape': (784,),
   'output_size': 784,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': None,
   'metadata': None},
  {'layer_type': <ConverterLayerType.DENSE: 0>,
   'input_shapes': ((784,),),
   'output_shape': (10,),
   'output_size': 10,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': {'weight': 0},
   'metadata': {'has_bias': False}},
  {'layer_type': <ConverterLayerType.OUTPUT: 7>,
   'input_shapes': ((10,),),
   'output_shape': (10,),
   'output_size': 10,
   'stack_input_indices': (0,),
   'stack_output_index': 0,
   'parameters': None,
   'metadata': None}]}

In [26]:
with open("test.coe", 'w') as f:
    f.write(converted_nn.generate_parameter_coe())

In [27]:
emulator = FPGAEmulator(converted_nn, bram_reserved_size=303000)

In [28]:
emulator.exec_info

{'input_shape': (1, 28, 28),
 'inital_input_addr': 0,
 'layers': [{'layer_type': <LayerType.DENSE: 0>,
   'config': {'has_bias': None,
    'input_base_addr': 0,
    'weight_base_addr': 0,
    'bias_base_addr': 0,
    'output_base_addr': 784,
    'm_size': 10,
    'chw_size': 784}},
  {'layer_type': <LayerType.MOVE: 5>,
   'config': {'input_base_addr': 784, 'output_base_addr': 0, 'n_size': 10}},
  {'layer_type': <LayerType.OUTPUT: 6>,
   'config': {'output_base_addr': 0, 'n_size': 10}}]}

In [29]:
for imgs, labels in dataloaders['test']:
    break
data = (imgs[0] * 255).numpy()
data = (data/2).astype(np.int8)
print(data)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  42  92  79  75  30  18   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0 111 127 127 127
 127 120  99  99  99  99  99  99  99  99  85  26   

In [30]:
parameters = converted_nn.parameters_info['parameters']
print([p.shape for p in parameters])

def dense_no_bias(w, i):
    pass

def relu(i):
    return np.maximum(0, i)

[(10, 784)]


In [31]:
inputs_per_layer = {}
outputs_per_layer = {}
def get_output(name):
    def hook(model, input, output):
        inputs_per_layer[name] = input
        outputs_per_layer[name] = output.detach()
    return hook

hooks = {}
for name, module in model_int8.named_modules():
    hooks[name] = module.register_forward_hook(get_output(name))

In [32]:
batch = imgs[None, 0, :]
model_out = model_int8(batch)

In [33]:
outputs_per_layer.keys()

dict_keys(['quant', 'model.layers.0', 'model.layers', 'model', 'dequant', ''])

In [34]:
inputs_per_layer['model.layers.0'][0].int_repr().numpy()

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [35]:
outputs_per_layer['model.layers.0'].int_repr()

tensor([[59, 33, 58, 68, 52, 54, 39, 98, 59, 71]], dtype=torch.uint8)

In [36]:
data

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [37]:
model_int8.model.layers[0].weight().int_repr()

tensor([[  7, -11, -13,  ...,  -6,   9,   6],
        [ -4,  -8,  -3,  ...,  -6,  -3,  -5],
        [ 14, -14, -14,  ...,   0,  -1,  10],
        ...,
        [ -5, -11,   7,  ...,  -3,   8,  -9],
        [ 16,   7,  12,  ...,  -4,  12, -13],
        [ -8,   2,   3,  ...,   7,  10,   0]], dtype=torch.int8)

In [38]:
(parameters[0] @ data).astype(np.uint8)

array([ 55, 185, 211,  60,  88,  71,  24,  69,  96, 112], dtype=uint8)

In [39]:
emulator.execute(data)

{'has_bias': None, 'input_base_addr': 0, 'weight_base_addr': 0, 'bias_base_addr': 0, 'output_base_addr': 784, 'm_size': 10, 'chw_size': 784}


C:\Users\Shahir\Documents\Code Repositories\6111-fpga-final-project\fpga_nn_backend\fpga_simple\emulation.py:176: RuntimeWarning: overflow encountered in byte_scalars
  o_out[i] = np.int8(w_in[i] * i_in[i] + b_in[i])


{'input_base_addr': 784, 'output_base_addr': 0, 'n_size': 10}
{'output_base_addr': 0, 'n_size': 10}


array([ 55, -71, -45,  60,  88,  71,  24,  69,  96, 112], dtype=int8)

In [40]:
np.where(data > 0)

(array([202, 203, 204, 205, 206, 207, 230, 231, 232, 233, 234, 235, 236,
        237, 238, 239, 240, 241, 242, 243, 244, 245, 258, 259, 260, 261,
        262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 291,
        292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 326, 327, 328,
        329, 353, 354, 355, 356, 381, 382, 383, 384, 408, 409, 410, 411,
        436, 437, 438, 439, 463, 464, 465, 466, 491, 492, 493, 518, 519,
        520, 521, 545, 546, 547, 548, 572, 573, 574, 575, 576, 600, 601,
        602, 603, 627, 628, 629, 630, 655, 656, 657, 658, 682, 683, 684,
        685, 686, 710, 711, 712, 713, 714, 738, 739, 740, 741], dtype=int64),)

In [41]:
parameters[0].flatten()[202]

-17

In [42]:
np.int8(-37*92)

-76

In [43]:
torch.quantization.get_default_qconfig('fbgemm')

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})

In [44]:
qscheme=torch.per_tensor_affine